In [1]:
import os.path
import pickle
import re
import json

import pandas as pd


In [2]:
def split_line_desc(df):
    """
    takes the Lines_desc feature and divides it to route_id, direction and alternative features.
    """
    
    # split LINE_DESC column by '-' and take the 0th element to create 'ID' column
    df['RouteId'] = df['LINE_DESC'].str.split('-').str[0]
    
    # split LINE_DESC column by '-' and take the 1st element to create 'DIRECTION' column
    df['Direction'] = df['LINE_DESC'].str.split('-').str[1]
    
    # split LINE_DESC column by '-' and take the 1st element to create 'DIRECTION' column
    df['Alternative'] = df['LINE_DESC'].str.split('-').str[2]
    return df

In [131]:
def add_columns_from_json(row):
    """
    splits the jason column to different features and adds them to the dataframe
    """
    global bad_jsons # counts the number of lines where the jason fails
    global no_previous
    global rows_list
    global test
    
    json_data = json.loads(row['jsonString'])
            
    for i,record in enumerate(json_data['tripMessages']):
        if i == 0:
            test = record
        try:
            for j,report in enumerate(record['tripMessage']):
                try:
                    if report['previousCalls']:
                        try:
                            actualArrivalTime = report['previousCalls'][0]['actualArrivalTime']
                        except:
                            actualArrivalTime = record['tripMessage'][0]['aimedDepartureTime']
                            if report['previousCalls'][0]['stopOrder'] != 1:
                                ERROR = 0
                                # raises an assertion if the stop order doesn't equal to 1.
                                assert(0)
                        
                        row_data = {
                                    "DatedVehicleJourneyRef": row['DatedVehicleJourneyRef'],
                                    "LINE_DESC": row["LINE_DESC"],
                                    "OPERATOR_ID": row["OPERATOR_ID"],
                                    "CLUSTER_ID": row["CLUSTER_ID"],
                                    "LINE_SHORT_NAME": row["LINE_SHORT_NAME"],
                                    "OriginAimedDepartureTime": row["OriginAimedDepartureTime"],
                                    "RouteId": row["RouteId"],
                                    "Direction": row["Direction"],
                                    "Alternative": row["Alternative"],
                                    "stopOrder": report['previousCalls'][0]['stopOrder'],
                                    "stopId": report['previousCalls'][0]['stopId'],
                                    "actualArrivalTime": actualArrivalTime,
                                    "actualDepartureTime": report['previousCalls'][0]['actualDepartureTime'],
                                    "License_Plate": row['VehicleRef']
                                   }
                        rows_list.append(row_data)
                except:
                    assert(ERROR)
                    #not an error not all entry's have the section previousCalls
        except:
            assert(ERROR)
            print(f'bad json {bad_jsons}')
            bad_jsons += 1

### opening a single vm file

In [3]:
if not os.path.isfile('examplePickle'):
    # Read the CSV file
    df = pd.read_csv(r"C:\Users\shalt\Documents\final project\data\VM_feb2023_1\VM_feb2023_1.csv", encoding="utf-8")
    dbfile = open('examplePickle', 'ab')
    # source, destination
    pickle.dump(df, dbfile)                     
    dbfile.close()
else:
    dbfile = open('examplePickle', 'rb')     
    df = pickle.load(dbfile)

df.shape

(21057, 11)

In [7]:
df

,OPERATOR_ID,CLUSTER_ID,LINE_DESC,LINE_SHORT_NAME,DatedVehicleJourneyRef,OriginAimedDepartureTime,VehicleRef,jsonString,RouteId,Direction,Alternative
0,3,91,13016-1-ר,16,5489309,2023-02-01 06:50:00.000,7584269,"{""tripMessages"":[{""tripMessage"":[{""messageCrea...",13016,1,ר
1,3,91,13016-1-ר,16,5489311,2023-02-01 07:10:00.000,7796169,"{""tripMessages"":[{""tripMessage"":[{""messageCrea...",13016,1,ר
2,3,91,13016-1-ר,16,5489295,2023-02-01 07:30:00.000,7721069,"{""tripMessages"":[{""tripMessage"":[{""messageCrea...",13016,1,ר
3,3,91,13016-1-ר,16,5489323,2023-02-01 07:50:00.000,7783069,"{""tripMessages"":[{""tripMessage"":[{""messageCrea...",13016,1,ר
4,3,91,13016-1-ר,16,5489291,2023-02-01 08:10:00.000,7790569,"{""tripMessages"":[{""tripMessage"":[{""messageCrea...",13016,1,ר
...,...,...,...,...,...,...,...,...,...,...,...
21052,3,91,46006-3-א,6א,585079446,2023-02-16 23:25:00.000,7552969,"{""tripMessages"":[{""tripMessage"":[{""messageCrea...",46006,3,א
21053,3,91,46006-3-א,6א,585079447,2023-02-16 23:30:00.000,7559969,"{""tripMessages"":[{""tripMessage"":[{""messageCrea...",46006,3,א
21054,3,91,46006-3-א,6א,585079448,2023-02-16 23:35:00.000,7804169,"{""tripMessages"":[{""tripMessage"":[{""messageCrea...",46006,3,א
21055,3,91,46006-3-א,6א,585079449,2023-02-16 23:40:00.000,7561869,"{""tripMessages"":[{""tripMessage"":[{""messageCrea...",46006,3,א


In [5]:
df = split_line_desc(df)

In [6]:
columns_to_drop = ['LINE_ID','DataFrameRef', 'RECORD_ID']
df = df.drop(columns_to_drop, axis=1)
df.columns

Index(['OPERATOR_ID', 'CLUSTER_ID', 'LINE_DESC', 'LINE_SHORT_NAME',
       'DatedVehicleJourneyRef', 'OriginAimedDepartureTime', 'VehicleRef',
       'jsonString', 'RouteId', 'Direction', 'Alternative'],
      dtype='object')

In [132]:
bad_jsons = 0
no_previous = 0
df_columns = list(df.columns)
column_names = df_columns + ['stopOrder', 'stopId', 'actualArrivalTime', 'actualDepartureTime']
print(column_names)
rows_list = []
test = 0

['OPERATOR_ID', 'CLUSTER_ID', 'LINE_DESC', 'LINE_SHORT_NAME', 'DatedVehicleJourneyRef', 'OriginAimedDepartureTime', 'VehicleRef', 'jsonString', 'RouteId', 'Direction', 'Alternative', 'stopOrder', 'stopId', 'actualArrivalTime', 'actualDepartureTime']


In [134]:
df.sample(2).apply(add_columns_from_json, axis=1)
print('apply finished')

bad json 0
apply finished


In [5]:
df_to_save = pd.DataFrame(rows_list)
df_to_save.drop_duplicates(inplace=True)
df_to_save = df_to_save.sort_values(by=['ID', 'OriginAimedDepartureTime', 'stopOrder'])
df_to_save.to_csv('1.csv')         

['OPERATOR_ID', 'CLUSTER_ID', 'LINE_SHORT_NAME', 'DatedVehicleJourneyRef', 'OriginAimedDepartureTime', 'jsonString', 'ID', 'stopOrder', 'stopId', 'actualArrivalTime', 'actualDepartureTime']
bad json 0
bad json 1
bad json 2
bad json 3
bad json 4
bad json 5
bad json 6
bad json 7
bad json 8
bad json 9
bad json 10
bad json 11
bad json 12
bad json 13
bad json 14
bad json 15
bad json 16
bad json 17
bad json 18
bad json 19
bad json 20
bad json 21
bad json 22
bad json 23
bad json 24
bad json 25
bad json 26
bad json 27
bad json 28
bad json 29
bad json 30
bad json 31
bad json 32
bad json 33
bad json 34
bad json 35
bad json 36
bad json 37
bad json 38
bad json 39
bad json 40
bad json 41
bad json 42
bad json 43
bad json 44
bad json 45
bad json 46
bad json 47
bad json 48
bad json 49
bad json 50
bad json 51
bad json 52
bad json 53
bad json 54
bad json 55
bad json 56
bad json 57
bad json 58
bad json 59
bad json 60
bad json 61
bad json 62
bad json 63
bad json 64
bad json 65
bad json 66
bad json 67
bad 

In [6]:
if not os.path.isfile('examplePickle1'):
    # Read the CSV file
    df1 = pd.read_csv('1.csv', index_col=0, encoding="utf-8")
    dbfile1 = open('examplePickle1', 'ab')
    # source, destination
    pickle.dump(df, dbfile1)                     
    dbfile1.close()
else:
    dbfile1 = open('examplePickle1', 'rb')     
    df1 = pickle.load(dbfile1)

df1.shape

C:\Users\davidfis\AppData\Local\Temp\ipykernel_6236\1644291577.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv('1.csv', index_col=0, encoding="utf-8")


(748355, 10)

In [7]:
#df1 = df1.loc[:, ~df1.columns.str.contains('^Unnamed')]
df1 = df1.reset_index(drop=True)
list(df1.columns)

['DatedVehicleJourneyRef',
 'OPERATOR_ID',
 'CLUSTER_ID',
 'LINE_SHORT_NAME',
 'OriginAimedDepartureTime',
 'ID',
 'stopOrder',
 'stopId',
 'actualArrivalTime',
 'actualDepartureTime']

In [8]:
rows_list_1 = []
column_names1 = ['Timestamp', 'Linkref', 'time_first_stop(s)', 'Link_travel_time(s)']

prev_row = None
output_rows = []

for index, row in df1.iterrows():
    if prev_row is not None and row['OriginAimedDepartureTime'] == prev_row['OriginAimedDepartureTime'] and prev_row['stopOrder']+1 == row['stopOrder']:
        # Calculate the link travel time in seconds
        prev_time = pd.to_datetime(prev_row['actualDepartureTime'])
        curr_time = pd.to_datetime(row['actualArrivalTime'])
        link_travel_time = int((curr_time - prev_time).total_seconds())

        # Calculate the time spent at the previous stop in seconds
        prev_arrival_time = pd.to_datetime(prev_row['actualArrivalTime'])
        prev_departure_time = pd.to_datetime(prev_row['actualDepartureTime'])
        time_in_stop = int((prev_departure_time - prev_arrival_time).total_seconds())

        # Create a new dictionary with the calculated data
        output_row = {
            "TripId": row['DatedVehicleJourneyRef'],
            "OPERATOR_ID": row["OPERATOR_ID"],
            "CLUSTER_ID": row["CLUSTER_ID"],
            "LINE_SHORT_NAME": row["LINE_SHORT_NAME"],
            "OriginAimedDepartureTime": row["OriginAimedDepartureTime"],
            "ID": row["ID"],
            "stopOrder": row['stopOrder'],
            "actualArrivalTime": row['actualArrivalTime'],
            "actualDepartureTime": row['actualDepartureTime'],
            'Linkref': f"{prev_row['stopId']}:{row['stopId']}",
            'linkTime': time_in_stop+link_travel_time,
            'time_first_stop(s)': time_in_stop,
            'Link_travel_time(s)': link_travel_time
        }

        # Add the new row to the output list
        output_rows.append(output_row)

    prev_row = row


print('iterrows finished')
df_to_save = pd.DataFrame(output_rows)
df_to_save.drop_duplicates(inplace=True)
df_to_save = df_to_save.sort_values(by=['ID', 'OriginAimedDepartureTime', 'stopOrder'])
df_to_save = df_to_save.reset_index(drop=True)
df_to_save.to_csv('2.csv')         

iterrows finished


In [9]:
dbfile = open('VM_1_Pickle', 'ab')
# source, destination
pickle.dump(df_to_save, dbfile)                     
dbfile.close()

# testing one record

In [135]:
def flatten_json(nested_json, parent_key='', sep='_'):
    flattened_dict = {}
    for key, value in nested_json.items():
        new_key = parent_key + sep + key if parent_key else key
        if isinstance(value, dict):
            flattened_dict.update(flatten_json(value, new_key, sep=sep))
        else:
            flattened_dict[new_key] = value
    return flattened_dict

In [136]:
test2 = flatten_json(test)
test3 = pd.json_normalize(test2, max_level=1).tripMessage[0]

In [137]:
test3 = pd.json_normalize(test3, max_level=2)
test3

,messageCreationTime,messageRecordingTime,messageReceiveTime,coordinateX,coordinateY,bearing,operatorId,vehicleAtStop,stopId,stopOrder,aimedDepartureTime,velocity,distanceFromStart,percentageFromStart,actualArrivalTime,previousCalls,actualDepartureTime,tripEndReasonCode
0,2023-02-01T06:02:00,2023-02-01T06:01:52,2023-02-01T06:02:03,35.179825,31.786882,290.0,3,True,230,1,2023-02-01T06:03:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-02-01T06:02:15,2023-02-01T06:01:52,2023-02-01T06:02:16,35.179825,31.786882,290.0,3,True,230,1,2023-02-01T06:03:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-02-01T06:02:30,2023-02-01T06:01:52,2023-02-01T06:02:31,35.179825,31.786882,290.0,3,True,230,1,2023-02-01T06:03:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-02-01T06:02:45,2023-02-01T06:01:52,2023-02-01T06:02:46,35.179825,31.786882,290.0,3,True,230,1,2023-02-01T06:03:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-02-01T06:03:00,2023-02-01T06:01:52,2023-02-01T06:03:02,35.179825,31.786882,290.0,3,True,230,1,2023-02-01T06:03:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154,2023-02-01T06:42:00,2023-02-01T06:41:34,2023-02-01T06:42:03,35.229473,31.797943,70.0,3,False,9807,61,NaN,22.0,15471.0,97.0,2023-02-01T06:40:54,"[{'previousCall': '1', 'stopId': 6136, 'stopOr...",2023-02-01T06:41:28,NaN
155,2023-02-01T06:42:15,2023-02-01T06:42:06,2023-02-01T06:42:16,35.230457,31.797997,8.0,3,False,9807,61,NaN,NaN,15594.0,98.0,2023-02-01T06:40:54,"[{'previousCall': '1', 'stopId': 6136, 'stopOr...",2023-02-01T06:41:28,NaN
156,2023-02-01T06:42:30,2023-02-01T06:42:14,2023-02-01T06:42:31,35.231010,31.798183,52.0,3,False,9807,61,NaN,25.0,15652.0,98.0,2023-02-01T06:40:54,"[{'previousCall': '1', 'stopId': 6136, 'stopOr...",2023-02-01T06:41:28,NaN
157,2023-02-01T06:42:45,2023-02-01T06:42:38,2023-02-01T06:42:46,35.231365,31.798538,58.0,3,False,9807,61,NaN,NaN,15704.0,99.0,2023-02-01T06:40:54,"[{'previousCall': '1', 'stopId': 6136, 'stopOr...",2023-02-01T06:41:28,NaN


In [140]:
pd.json_normalize(test3.previousCalls[158])

,previousCall,stopId,stopOrder,actualArrivalTime,actualDepartureTime
0,1,9807,61,2023-02-01T06:40:54,2023-02-01T06:41:28
1,2,6136,60,2023-02-01T06:40:10,2023-02-01T06:40:26
